In [1]:
import pytorch_lightning as pl
import torch 
import torch.nn as nn 

from torchmetrics import __version__ as torchmetrics_version
from pkg_resources import parse_version

from torchmetrics import Accuracy

### Lightning class

In [60]:
class MultiLayerPerceptron(pl.LightningModule):
    def __init__(self, image_shape = (1,28,28), hidden_units = (32,16)):
        super().__init__()
        self.train_acc = Accuracy(task = "multiclass", num_classes = 10)
        self.valid_acc = Accuracy(task = "multiclass", num_classes = 10)
        self.test = Accuracy(task = "multiclass", num_classes = 10)
        
        input_size = image_shape[0]*image_shape[1]*image_shape[2]
        all_layers = [nn.Flatten()]
        
        for hidden_unit in hidden_units:
            layer = nn.Linear(input_size, hidden_unit)
            all_layers.append(layer)
            all_layers.append(nn.ReLU())
            input_size = hidden_unit
        
        all_layers.append(nn.Linear(hidden_units[-1], 10))
        self.model = nn.Sequential(*all_layers)

    
    def forward(self,x):
        x = self.model(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x,y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits,y)
        preds = torch.argmax(logits, dim = 1 )
        self.train_acc.update(preds, y)
        self.log("train_loss", loss, prog_bar = True)
        return loss
    
    def train_epoch_end(self,outs):
        self.log("train_acc", self.train_acc.compute())
        self.train_acc.reset()
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.valid_acc.update(preds, y)
        self.log("valid_loss", loss, prog_bar=True)
        return loss
    
    def on_validation_epoch_end(self, outs):
        self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
        self.valid_acc.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_acc.compute(), prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer
    
        

In [61]:
model = MultiLayerPerceptron()

In [50]:
model

MultiLayerPerceptron(
  (train_acc): MulticlassAccuracy()
  (valid_acc): MulticlassAccuracy()
  (test): MulticlassAccuracy()
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ReLU()
    (5): Linear(in_features=16, out_features=10, bias=True)
  )
)

In [26]:
x = torch.rand(10,1*28,28)

In [44]:
x = torch.rand(2,3)

In [45]:
x

tensor([[0.3420, 0.2535, 0.7935],
        [0.0587, 0.8880, 0.5211]])

In [46]:
torch.argmax(x, dim = 1)

tensor([2, 1])

In [52]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split
 
from torchvision.datasets import MNIST
from torchvision import transforms

## Prepare dataset

In [56]:
class MnistDataModule(pl.LightningDataModule):
    def __init__(self,data_path = "./"):
        super().__init__()
        self.data_path = data_path
        self.transform = transforms.Compose([transforms.ToTensor()])
      
    def prepare_data(self):
        MNIST(root  = self.data_path, download = True)
    
    def setup(self, stage= None):
        mnist_all = MNIST(
        root = self.data_path,
            train = True,
            transform = self.transform,
            download = False
        )
        
        self.train, self.val = random_split(
        mnist_all, [55000, 5000], generator = torch.Generator().manual_seed(1)
        )
        
        self.test = MNIST(
        root = self.data_path,
        train = False,
            transform = self.transform,
            download= False
        )

    def train_dataloader(self):
        return DataLoader(self.train, batch_size = 64, num_workers = 4)
    
    def val_dataloader(self):
        return DataLoader(self.val, batch_size=64, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=64, num_workers=4)
  

torch.manual_seed(1)
mnist_dm = MnistDataModule()


In [58]:
from pytorch_lightning.callbacks import ModelCheckpoint


In [63]:
mnistclassifier = MultiLayerPerceptron()

callbacks = [ModelCheckpoint(save_top_k=1, mode='max', monitor="valid_acc")] # save top 1 model

trainer = pl.Trainer(max_epochs = 10, callbacks = callbacks)
trainer.fit(model = mnistclassifier, datamodule = mnist_dm)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


100%|████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:19<00:00, 511998.15it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|███████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 5089093.55it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



...

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 4606027.26it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw





  | Name      | Type               | Params
-------------------------------------------------
0 | train_acc | MulticlassAccuracy | 0     
1 | valid_acc | MulticlassAccuracy | 0     
2 | test      | MulticlassAccuracy | 0     
3 | model     | Sequential         | 25.8 K
-------------------------------------------------
25.8 K    Trainable params
0         Non-trainable params
25.8 K    Total params
0.103     Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.41s/it]

TypeError: MultiLayerPerceptron.on_validation_epoch_end() missing 1 required positional argument: 'outs'

In [65]:
import sys
sys.path.insert(0, '..')

In [66]:
from python_environment_check import check_packages


d = {
    'torch': '1.8',
    'torchvision': '0.9.0',
    'tensorboard': '2.7.0',
    'pytorch_lightning': '1.5.0',
    'torchmetrics': '0.6.2'
}
check_packages(d)

ModuleNotFoundError: No module named 'python_environment_check'

In [67]:
ls

Deep_mechanics_of_pytorch.ipynb
MNIST/
Pytorch Lightning.ipynb
cat_dog_images/
lightning_logs/
predicting_fuel_efficiency_of_car.ipynb
pytorch_basics_and_simple_multilayer_perceptron.ipynb


In [68]:
pytorch_lightning --version

NameError: name 'pytorch_lightning' is not defined

In [69]:
pl.__version__

'2.0.5'